In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 간단한 MLP 모델
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(10, 16)  # dead neuron이 잘 발생하는 구조
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        z = self.fc1(x)
        a = self.relu(z)  # 죽은 뉴런 확인할 부분
        out = self.fc2(a)
        return out, z, a


model = SimpleMLP()

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# 2. 음수 위주의 입력 데이터 → Dead neuron 발생 증가
X = torch.randn(200, 10) - 2.0   # 음수로 bias
y = torch.randn(200, 1)

# 3. 뉴런이 죽었는지 확인하는 함수
def check_dead_neurons(activation):
    # activation shape: (batch_size, num_neurons)
    dead_ratio = (activation == 0).float().mean(dim=0)
    print("각 뉴런의 Zero Activation 비율:")
    print(dead_ratio)


# 4. 학습 전 Dead neuron 체크
model.eval()
with torch.no_grad():
    _, z, a = model(X)
    print("🔥 학습 전 Dead Neuron 비율")
    check_dead_neurons(a)

# 5. 학습 진행
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    out, _, a = model(X)
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()

# 6. 학습 후 Dead neuron 체크
model.eval()
with torch.no_grad():
    _, z, a = model(X)
    print("\n🔥 학습 후 Dead Neuron 비율")
    check_dead_neurons(a)


🔥 학습 전 Dead Neuron 비율
각 뉴런의 Zero Activation 비율:
tensor([0.0100, 0.9250, 0.0550, 0.9500, 0.5400, 0.9150, 1.0000, 0.8750, 0.3350,
        0.9900, 0.3100, 0.0250, 0.5250, 0.2800, 0.6050, 0.5250])

🔥 학습 후 Dead Neuron 비율
각 뉴런의 Zero Activation 비율:
tensor([0.0000, 0.7900, 0.2850, 0.9000, 0.6800, 0.6000, 1.0000, 1.0000, 0.9150,
        0.9800, 0.1400, 0.0100, 0.1900, 0.8100, 0.8050, 0.9900])
